# Data Preparation

## Setup

In [1]:
# Import useful libraries
import numpy as np
import pandas as pd
import os
import re
import pickle
import sys
import time
import psycopg2

In [2]:
# Setup Repository
with open("repo_info.txt", "r") as repo_info:
    path_to_repo = repo_info.readline()

path_to_data = f"{path_to_repo}data/"
path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"

In [4]:
# Read from a txt file the information on the credentials for our local PgAdmin server
with open("pg_admin_info.txt", "r") as repo_info:
    pg_credentials = repo_info.readlines()

sqluser = pg_credentials[0].strip()
dbname = pg_credentials[1].strip()
schema_name =  pg_credentials[2].strip()
password = pg_credentials[3].strip()

In [12]:
# PARAMETERS -----------------------------------
# Set to True if we want to include deaths
death_incl = True
death_tag = np.where(death_incl,"_death", "")

## Import Data

In [13]:
df = pd.read_csv(f'{path_to_raw}all_data_raw.csv.gzip', compression = 'gzip')

C:\Users\luca9\AppData\Local\Temp\ipykernel_15712\2730209426.py:1: DtypeWarning: Columns (4,13,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{path_to_raw}all_data_raw.csv.gzip', compression = 'gzip')


In [14]:
# Then we drop any duplicates
df = df.drop_duplicates(subset=['hadm_id']).reset_index()

In [15]:
# And print info our dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36141 entries, 0 to 36140
Data columns (total 68 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               36141 non-null  int64  
 1   hadm_id             36141 non-null  int64  
 2   subject_id          36141 non-null  int64  
 3   admittime           36141 non-null  object 
 4   dischtime           36141 non-null  object 
 5   deathtime           4693 non-null   object 
 6   ethnicity           36141 non-null  object 
 7   admission_type      36141 non-null  object 
 8   admission_location  36141 non-null  object 
 9   insurance           36141 non-null  object 
 10  religion            36019 non-null  object 
 11  marital_status      34445 non-null  object 
 12  discharge_location  36141 non-null  object 
 13  costcenter          36141 non-null  object 
 14  cpt_code            36141 non-null  object 
 15  first_careunit      36141 non-null  object 
 16  last

In [16]:
df.head()

,index,hadm_id,subject_id,admittime,dischtime,deathtime,ethnicity,admission_type,admission_location,insurance,...,icd9_code,proc_icd9,diag_icd9,age_cat,type_stay,prev_adm,dest_discharge,emergency_dpt,icd_chapter,origin_patient
0,0,100003,54610,2150-04-17 15:34:00,2150-04-21 17:30:00,NaN,WHITE,EMERGENCY,EMERGENCY ROOM ADMIT,Private,...,53100,"4443,9607,9904,3893","53100,2851,07054,5715,45621,53789,4019,53550,7823",3,1-Medical,1-No hospitalization,1-Home,Yes,9,2-Other
1,6,100006,9895,2108-04-06 15:49:00,2108-04-18 17:18:00,NaN,BLACK/AFRICAN AMERICAN,EMERGENCY,EMERGENCY ROOM ADMIT,Private,...,49320,"9390,9925","49320,51881,486,20300,2761,7850,3090,V1251,V1582",3,1-Medical,1-No hospitalization,1-Home,Yes,8,2-Other
2,14,100007,23018,2145-03-31 05:33:00,2145-04-07 12:40:00,NaN,WHITE,EMERGENCY,EMERGENCY ROOM ADMIT,Private,...,56081,"4562,5459","56081,5570,9973,486,4019",4,3-Surgical,1-No hospitalization,1-Home,Yes,9,2-Other
3,17,100009,533,2162-05-16 15:56:00,2162-05-21 13:37:00,NaN,WHITE,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Private,...,41401,"3613,3615,3795,3961","41401,99604,4142,25000,27800,V8535,4148,4111,V...",3,3-Surgical,1-No hospitalization,1-Home,Yes,7,2-Other
4,25,100010,55853,2109-12-10 07:15:00,2109-12-14 16:45:00,NaN,WHITE,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Private,...,1890,"5551,540,403","1890,1961,1987,1976,27652",3,1-Medical,1-No hospitalization,1-Home,No,2,1-Home


In [17]:
# Convert the deathtime column to a datetime-column
df['deathtime'] = pd.to_datetime(df.deathtime, errors = 'coerce')
# Remove any observation with a recorded death case
if death_incl == False:
    df_clean = df[np.isnat(df['deathtime'])].copy()
    print(f"Number of observations with a deathtime: {df.shape[0] - df_clean.shape[0]}")
else:
    df_clean = df.copy()

In [18]:
# We then remove all patients below 18 years old
df_clean = df_clean.loc[df_clean.age >= 18]
print(f"N. of observations: {df_clean.shape[0]}")

N. of observations: 36078


### Process NOTEEVENTS to get discharge notes

In [19]:
# SQL query (gets all_data view defined elsewhere)
query = \
"""
select * from note_filter
"""

In [20]:
# Connect to local postgres version of mimic
con = psycopg2.connect(host='localhost', database=dbname, user=sqluser, password=password)
cur = con.cursor()

In [22]:
# Perform SQL query
cur.execute('SET search_path to ' + schema_name)
text = pd.read_sql_query(query,con)

In [24]:
# Remove any duplicates
text.drop_duplicates(inplace = True)

In [25]:
text.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,first_careunit,last_careunit,age,gender,marital_status,insurance,diagnosis,text,iserror
0,22532,167853,2151-07-16 14:29:00,2151-08-04 19:10:00,NaT,MICU,MICU,86.902940,F,UNKNOWN (DEFAULT),Medicare,RT LOWER LOBE PNEUMONIA,Admission Date: [**2151-7-16**] Dischar...,None
1,13702,107527,2118-06-02 19:18:00,2118-06-14 15:15:00,NaT,MICU,MICU,81.082691,F,MARRIED,Medicare,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,Admission Date: [**2118-6-2**] Discharg...,None
2,13702,167118,2119-05-04 07:15:00,2119-05-25 13:30:00,NaT,CSRU,MICU,82.001254,F,MARRIED,Medicare,TRACHEALBRONCHEAL MALACEA/SDA,Admission Date: [**2119-5-4**] D...,None
3,13702,196489,2124-07-21 16:09:00,2124-08-18 16:26:00,NaT,MICU,MICU,87.217989,F,MARRIED,Medicare,ASTHMA;COPD EXACERBATION,Admission Date: [**2124-7-21**] ...,None
4,26880,135453,2162-03-03 18:46:00,2162-03-25 14:15:00,NaT,TSICU,TSICU,82.161367,M,MARRIED,Medicare,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,None


Our discharge notes are not unique for the combination subject_id and hadm_id. We will merge all the discharge notes.

In [26]:
print(f"Number of duplicate text: {round(text.duplicated(subset = ['subject_id', 'hadm_id']).sum()/text.shape[0]*100, 2)}%")

Number of duplicate text: 14.6%


In [27]:
text['text'] = text.groupby(['subject_id', 'hadm_id', 'admittime']).text.transform(lambda x: '\n'.join(x))

In [28]:
text_clean = text[['hadm_id', 'subject_id', 'text']].drop_duplicates()
print(f"Duplicates removed: {text.shape[0] - text_clean.shape[0]}")

Duplicates removed: 8345


## Merge Text with Tabular Data

In [29]:
df_full = pd.merge(df_clean, text_clean, on = ['hadm_id', 'subject_id'], indicator = True, how = 'left', validate = '1:1')
print(f"Check merges:\n{df_full._merge.value_counts()}")

Check merges:
both          35268
left_only       810
right_only        0
Name: _merge, dtype: int64


In [30]:
# Now remove the failed merges
df_full = df_full.loc[df_full._merge == 'both']
df_full.drop(columns = ['_merge'], inplace = True)

In [31]:
# Finally save the file to a CSV
df_full.to_csv(f'{path_to_processed}df_mixed{death_tag}.csv.gzip', compression = 'gzip', index = False)